# Day 3: Gear Ratios

## Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (`.`) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: `114` (top right) and `58` (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is `4361`.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [3]:
import numpy as np

test_data = np.array([
    list(line)
    for line in [
        "467..114..",
        "...*......",
        "..35..633.",
        "......#...",
        "617*......",
        ".....+.58.",
        "..592.....",
        "......755.",
        "...$.*....",
        ".664.598..",
    ]
])

test_data

array([['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']], dtype='<U1')

In [4]:
import itertools
import copy

DIRECTIONS = [
    direction 
    for direction in itertools.product([-1, 0, 1], repeat=2)
    if direction != (0, 0)
]

def part1(data):
    def is_adjacent(x, y):
        def is_symbol(s):
            return (not s.isdigit() and s != ".")
        return any([
            is_symbol(data[x + x_dir, y + y_dir])
            for x_dir, y_dir in DIRECTIONS
            if 0 <= x + x_dir < data.shape[0] and 0 <= y + y_dir < data.shape[1]
        ])

    result = 0

    DEFAULT_NUMBER = ("", False) # (substr, is_adjacent)
    number = copy.deepcopy(DEFAULT_NUMBER)

    for x, y in np.ndindex(*data.shape):
        char = data[x, y]
        if (not char.isdigit()) or (y == 0):
            if number[1] and len(number[0]) > 0:
                result += int(number[0])
            number = copy.deepcopy(DEFAULT_NUMBER)
        
        if char.isdigit():
            number = (number[0] + char, number[1] or is_adjacent(x, y))
    
    return result

part1(test_data)

4361

In [5]:
with open("input.txt") as f:
    data = np.array([list(line.strip()) for line in f.readlines()])

data[:5, :5]

array([['.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.'],
       ['.', '.', '.', '3', '7'],
       ['.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.']], dtype='<U1')

In [6]:
part1(data)

550934

## Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any `*` symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers `467` and `35`, so its gear ratio is `16345`. The second gear is in the lower right; its gear ratio is `451490`. (The `*` adjacent to `617` is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces `467835`.

What is the sum of all of the gear ratios in your engine schematic?



In [7]:
def part2(data):
    def get_expanded_to_left_str(x, y):
        result = ""
        while y >= 0 and data[x, y].isdigit():
            result = data[x, y] + result
            y -= 1
        return result
    def get_expanded_to_right_str(x, y):
        result = ""
        while y < data.shape[1] and data[x, y].isdigit():
            result += data[x, y]
            y += 1
        return result
    def get_expanded_number(x, y):
        number_str = get_expanded_to_left_str(x, y - 1) + get_expanded_to_right_str(x, y)
        return int(number_str) if len(number_str) > 0 else None

    result = 0
    for x, y in zip(*np.where(data == "*")):
        adjacent = []

        for row in [x + x_dir for x_dir in [-1, 0, 1] if 0 <= x + x_dir <= data.shape[0] - 1]:
            number_to_left_str = get_expanded_to_left_str(row, y - 1)
            number_to_right_str = get_expanded_to_right_str(row, y + 1)
            for number_str in (number_to_left_str + data[row, y] + number_to_right_str).replace("*", ".").split("."):
                if len(number_str) > 0:
                    adjacent.append(int(number_str))

        if len(adjacent) == 2:
            result += np.prod(adjacent)

    return result
        

part2(test_data)

467835

In [8]:
part2(data)

81997870